# Student Loan Risk with Deep Learning

In [25]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [26]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [27]:
loans_df.columns

Index(['payment_history', 'location_parameter', 'stem_degree_score',
       'gpa_ranking', 'alumni_success', 'study_major_code',
       'time_to_completion', 'finance_workshop_score', 'cohort_ranking',
       'total_loan_score', 'financial_aid_score', 'credit_ranking'],
      dtype='object')

In [28]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [29]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [30]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [31]:
# Define features set X by selecting all columns but credit_ranking

X = loans_df.drop("credit_ranking", axis=1)
# Review the features DataFrame
len(X)

1599

### Step 3: Split the features and target sets into training and testing datasets.


In [32]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [43]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)

# Fit the scaler to the features training dataset
X_test_scaled = scaler.fit_transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [34]:
# Define the the number of inputs (features) to the model
input_features = len(X.columns)

# Review the number of features
input_features


11

In [35]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
output_neuron=1

In [36]:
# Create the Sequential model instance
nn_model = tf.keras.Sequential()

# Add the first hidden layer
nn_model.add(
  tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                        input_dim=input_features,
                        name = "H1",
                        activation="relu")
)

# Add the second hidden layer
nn_model.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2,
                          name = "H2",
                          activation="relu")
)

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=1, name="output", activation="sigmoid"))


c:\Users\dipes\anaconda_3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ H1 (Dense)                      │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ H2 (Dense)                      │ (None, 3)              │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [38]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [44]:
# Fit the model using 50 epochs and the training data
nn_fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.4940 - loss: 0.7159
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5670 - loss: 0.6779 
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6022 - loss: 0.6586
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6667 - loss: 0.6307
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7043 - loss: 0.6147
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6982 - loss: 0.6154
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7046 - loss: 0.5925
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7166 - loss: 0.5888
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7199 - loss: 0.5677
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7256 - loss: 0.5662
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7109 - loss: 0.5779
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7389 - 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [45]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 18ms/step - accuracy: 0.7425 - loss: 0.5056
Loss: 0.5055967569351196, Accuracy: 0.7425000071525574


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [46]:
# Set the model's file path
file_path = Path("saved_model/student_loans.keras")


# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [47]:
# Set the model's file path
file_path = Path("saved_model/student_loans.keras")

# Load the model to a new object
nn_model_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [51]:
# Make predictions with the test data
prediction = nn_model_imported.predict(X_test_scaled)


# Display a sample of the predictions
display(prediction[0:5])

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[0.24111201],
       [0.34933773],
       [0.90000695],
       [0.8011811 ],
       [0.98298156]], dtype=float32)

In [52]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=prediction).round()
predictions_df

,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [53]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.72      0.74      0.73       188
           1       0.76      0.75      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.74      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
Ideally, the model would have been further fine-tuned to improve loss metrics and accuracy. For the purpose of recommending student loan options, we need a system that can suggest loans for which students are likely to qualify. Our model, with an accuracy of ~75% in determining creditworthiness, requires us to collect data consistent with the features used in the model: payment history, location parameter, STEM degree score, GPA ranking, alumni success, study major code, time to completion, finance workshop score, cohort ranking, total loan score, financial aid score, and credit ranking. This data includes demographic information, financial history, and educational/institutional details, enabling us to recommend loans that students are likely to be approved for. Additionally, we could gather information on students’ loan preferences, such as terms and repayment penalties, to provide more customized recommendations.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
I would design a system to recommend student loan options will largely be based on collaborative filtering. It may have few content-based filtering. I do not expect to use context based filtering  

The recommendation system shall utilize inputs from user to compare with data fron other students having similar situation. I will suggest the content based filering in my model. The content-based components will  recommend specific loan products or structures based on loan features preffered by the student - for example, if the student is searching for "loans with no penalties for prepayment, the recocomendation will be for type of loans that the student will not only qualify but also will find it interesting. This will likely satisfy the user and convert the student to a loan customer. 

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
I foresee a challenge in gathering data for effective use in the model. Partly, due to privacy concerns and fairness. As an example  if a student has a particular class ranking or choice of major and minor should not have an impcact student's education? At the sanme time, we would not want disproportionate benefit to certain classes of students

The users (in this case students) cannot be trusted to enter the data correctly. You canno train a user tp input the data. We also need to keep in mind how will we obtain this data? Would it be sbmitted by schools? Schools can give you data with inflated numbers to over-estimate the value of their degrees.

A novel consideration that comes to mind. Instead of students always pursuing 4 year college, the model could look at options of trade schools which are being pursued regularly these days. These students could present an untapped opportunity for lenders that may be overlooked by lender community.
